<a href="https://colab.research.google.com/github/034adarsh/Blackcoffers_internship_task-Data-Extraction-and-NLP/blob/main/Blackcoffer_Internship.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Task-1

Data extraction from the given urls in the Input.xlsx file.
Instructions:

* Upload Input.xlsx file in the same directory

* Install the required libraries: openpyxl, requests, beautifulsoup4, and urllib.

* Import the necessary modules and libraries.

* Define the extract_article_text(file_path) function.

* Load the Excel file and get the active sheet.

* Iterate over each row in the sheet, extracting the URL ID and article URL.

* Call the extract_text_from_url(article_url) function to retrieve the article text.

* If the article_text is not None, save it to a text file using the URL ID as the file name.

* Define the extract_text_from_url(article_url) function.

* Send a GET request to the article URL and handle any HTTP errors.

Create a BeautifulSoup object to parse the HTML content.

* Find the article content element and extract the article title and paragraphs.

* Return the concatenated article title and text.

* Handle any exceptions and print error messages if necessary.

* Provide the file path of the input Excel file.

* Call the extract_article_text(input_file_path) function to start the extraction process.

In [1]:
import openpyxl
import requests
import urllib.error
from bs4 import BeautifulSoup

def extract_article_text(file_path):
    # Load the input.xlsx file
    wb = openpyxl.load_workbook(file_path)
    sheet = wb.active

    # Iterate over each row in the sheet
    for row in sheet.iter_rows(min_row=2, values_only=True):
        url_id, article_url = row

        # Extract article text from the article_url
        article_text = extract_text_from_url(article_url)

        # Save the extracted article text to a text file
        if article_text is not None:
            file_name = f"{url_id}.txt"
            with open(file_name, 'w', encoding='utf-8') as file:
                file.write(article_text)

def extract_text_from_url(article_url):
    try:
        # Send a GET request to the article URL
        response = requests.get(article_url)
        response.raise_for_status()

        # Create a BeautifulSoup object to parse the HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the article content element based on its HTML structure
        article_content = soup.find('div', class_='post-content')

        # Initialize article title and text as empty strings
        article_title = ''
        article_text = ''

        # Check if the article content element exists
        if article_content:
            # Extract article title if found
            article_title_element = article_content.find('h1')
            if article_title_element:
                article_title = article_title_element.get_text(strip=True)

            # Extract article text if found
            paragraphs = article_content.find_all('p')
            if paragraphs:
                article_text = ' '.join(paragraph.get_text(strip=True) for paragraph in paragraphs)

        # Return the concatenated article title and text
        return f"{article_title}\n\n{article_text}"

    except requests.exceptions.HTTPError as e:
        print(f"Error accessing URL: {article_url}")
        print(f"HTTP Error: {e}")
        return None
    except urllib.error.URLError as e:
        print(f"Error accessing URL: {article_url}")
        print(f"URL Error: {e}")
        return None

# Usage
input_file_path = 'Input.xlsx'
extract_article_text(input_file_path)

Error accessing URL: https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/
HTTP Error: 404 Client Error: Not Found for url: https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/
Error accessing URL: https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/
HTTP Error: 404 Client Error: Not Found for url: https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/
Error accessing URL: https://insights.blackcoffer.com/ensuring-growth-through-insurance-technology/
HTTP Error: 404 Client Error: Not Found for url: https://insights.blackcoffer.com/ensuring-growth-through-insurance-technology/


## Task-2
* Upload the Input.xlsx file and the Output Data Structure.xlsx file to your Google Drive. Make sure the files are in the same directory.

* Import the necessary libraries:

  1. pandas
  2. requests
  3. BeautifulSoup
  4. pyphen
  5. TextBlob
* Define the function perform_textual_analysis to perform the textual analysis and compute the desired variables.

* Specify the file paths for the input and output files.

* Call the function perform_textual_analysis with the provided input and output file paths.

* Run the code by selecting the code block and clicking the "Run" button or pressing Shift+Enter.

* Wait for the code to process the articles and perform textual analysis. The output will be saved in the specified Output Data Structure.xlsx file.

* Once the code execution is complete, you can download the updated Output Data Structure.xlsx file from your Google Drive.

In [36]:
!pip install openpyxl
!pip install nltk
!pip install textblob
!pip install syllables
%pip install pyphen
import nltk
nltk.download('stopwords')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [53]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pyphen
from textblob import TextBlob

def perform_textual_analysis(input_file_path, output_file_path):
    # Read input file
    input_df = pd.read_excel(input_file_path)

    # Initialize output dataframe
    output_df = pd.DataFrame(columns=[
        'URL_ID',
        'POSITIVE SCORE',
        'NEGATIVE SCORE',
        'POLARITY SCORE',
        'SUBJECTIVITY SCORE',
        'AVG SENTENCE LENGTH',
        'PERCENTAGE OF COMPLEX WORDS',
        'FOG INDEX',
        'AVG NUMBER OF WORDS PER SENTENCE',
        'COMPLEX WORD COUNT',
        'WORD COUNT',
        'SYLLABLE PER WORD',
        'PERSONAL PRONOUNS',
        'AVG WORD LENGTH'
    ])

    # Create Pyphen instance for syllable counting
    dic = pyphen.Pyphen(lang='en_US')

    # Process each URL
    for index, row in input_df.iterrows():
        url = row['URL']
        url_id = row['URL_ID']

        # Fetch article content from URL
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        article = soup.find('article')

        # Extract article text
        article_text = article.get_text() if article else ''

        # Perform textual analysis
        blob = TextBlob(article_text)
        sentences = blob.sentences
        words = blob.words
        word_count = len(words)

        # Calculate sentiment scores
        sentiment_scores = blob.sentiment
        polarity_score = sentiment_scores.polarity
        subjectivity_score = sentiment_scores.subjectivity
        positive_score = sentiment_scores.polarity if sentiment_scores.polarity > 0 else 0
        negative_score = -sentiment_scores.polarity if sentiment_scores.polarity < 0 else 0

        # Calculate average sentence length
        avg_sentence_length = len(words) / len(sentences) if len(sentences) > 0 else 0

        # Calculate percentage of complex words
        complex_words = [word for word in words if len(dic.positions(word)) >= 3]
        percentage_complex_words = (len(complex_words) / len(words)) * 100 if len(words) > 0 else 0

        # Calculate FOG index
        fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)

        # Calculate average number of words per sentence
        avg_words_per_sentence = len(words) / len(sentences) if len(sentences) > 0 else 0

        # Calculate complex word count
        complex_word_count = len(complex_words)

        # Calculate syllables per word
        syllables_per_word = sum(len(dic.positions(word)) + 1 for word in words) / word_count if word_count > 0 else 0

        # Calculate personal pronouns count
        personal_pronouns = ['I', 'you', 'he', 'she', 'it', 'we', 'they', 'me', 'him', 'her', 'us', 'them']
        personal_pronouns_count = sum(word.lower() in personal_pronouns for word in words)

        # Calculate average word length
        avg_word_length = sum(len(word) for word in words) / word_count if word_count > 0 else 0

        # Append results to output dataframe
        output_df = output_df.append({
            'URL_ID': url_id,
            'POSITIVE SCORE': positive_score,
            'NEGATIVE SCORE': negative_score,
            'POLARITY SCORE': polarity_score,
            'SUBJECTIVITY SCORE': subjectivity_score,
            'AVG SENTENCE LENGTH': avg_sentence_length,
            'PERCENTAGE OF COMPLEX WORDS': percentage_complex_words,
            'FOG INDEX': fog_index,
            'AVG NUMBER OF WORDS PER SENTENCE': avg_words_per_sentence,
            'COMPLEX WORD COUNT': complex_word_count,
            'WORD COUNT': word_count,
            'SYLLABLE PER WORD': syllables_per_word,
            'PERSONAL PRONOUNS': personal_pronouns_count,
            'AVG WORD LENGTH': avg_word_length
        }, ignore_index=True)

    # Save output dataframe to Excel file
    output_df.to_excel(output_file_path, index=False)

# Provide the input and output file paths
input_file_path = 'Input.xlsx'
output_file_path = 'Output Data Structure.xlsx'

# Call the function to perform textual analysis
perform_textual_analysis(input_file_path, output_file_path)


<ipython-input-53-6c7db187077f>:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({
<ipython-input-53-6c7db187077f>:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({
<ipython-input-53-6c7db187077f>:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({
<ipython-input-53-6c7db187077f>:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({
<ipython-input-53-6c7db187077f>:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df